
# **Slot Machine**<br>
If you are using BigQuery with the on demand billing model where you are billed by bytes scanned, it may be a good idea to switch to the Editions model where you are billed for slots you used.

There are Two quite complicated querstions though:


1.   Will moving to Editions save me money ?
2.   How many slots should I reserve, what is the optimal max_xlots value ?

There is no way to be 100% accurate, but this set of queries aims to reduce the guesswork and guide you in finding better answers to the Two questions above.





**Usage**<br>
You should run this in the project you want to test.
The process creates several tables and views, if any of them already exists then it will stop with error, since we don't want to accidently delete user's important tables.

**Step 1**<br>
First we set some parameters such as region, timeframe and BigQuery edition. We also detect the current project id.

Here is a short description of the parameters:

- **region**: The region where your dataset is located. INFORMATION_SCHEMA is a regional data source.
- **dataset**: The name of a new dataset where you want the tables and views related to slot machine to be created. Don't use an existing dataset.
- **start and end timestamp**: We analyze the behavior in a limited timeframe. It should be long enough to contain all regular activity so it should contain at lease few days.
- **verbose**: If true, Slot machine will print out the queries it runs plus additional interim data. If false it will operate quietly without printing out every query and interim result.

In [ ]:
import google.auth
project_id = google.auth.default()[1]
region = "US" # @param {"type":"string", "placeholder": "Enter dataset region"}
dataset = "test2" # @param {"type":"string", "placeholder": "Enter the target dataset where objects will be created"}
start_timestamp = "2024-12-01" # @param {"type":"date"}
end_timestamp = "2024-12-08" # @param {"type":"date"}
verbose = False # @param {"type":"boolean"}

**Step 2**<br>
Here we import some python packages and create the dataset if it does not already exist.

In [ ]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

client = bigquery.Client()
my_dataset = bigquery.Dataset(project_id+"."+dataset)
my_dataset.location = region
se_price = 0.04
ee_price = 0.06
eep_price = 0.1


try:
    client.get_dataset(project_id+"."+dataset)
    print("Dataset {} already exists".format(dataset))
except NotFound:
    print("Dataset {} is not found".format(dataset), "creating it.")
    dataset_object = client.create_dataset(my_dataset, timeout=30)

**Step 3**<br>
Ceate a table with the on demand consumption for reference. The table name is bytes.<br>
We select from INFORMATION_SCHEMA.JOBS to find the total bytes billed during the timeframe and what was the cost in USD.<br>Write down the result so you can compare it to the spend when using slots.

In [ ]:
print(dataset)
query = "create table if not exists "+dataset+".bytes as "+"""SELECT
  SUM(total_bytes_billed/1024/1024/1024/1024) AS total_tb_billed,
  SUM(total_bytes_billed/1024/1024/1024/1024)*6.25 as cost_usd
FROM """+project_id+".region-"+region+""".INFORMATION_SCHEMA.JOBS
WHERE
  creation_time BETWEEN CAST(\""""+start_timestamp+"""\" AS TIMESTAMP)
  AND """+"CAST(\""+end_timestamp+"\" AS TIMESTAMP)"

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

**Step 4**<br>
Then we query INFORMATION_SCHEMA.JOBS_TIMELINE on the same time range to see how many slots were used in every second of the time range. We send the result to a table called slots.

In [ ]:
query = "create table if not exists "+dataset+".slots as "+"""SELECT
  period_start,
  count (distinct job_id) as total_queries,
  SUM(period_slot_ms/1000) AS total_slot_ms,
FROM
  """+project_id+".region-"+region+""".INFORMATION_SCHEMA.JOBS_TIMELINE
WHERE
  period_start BETWEEN CAST(\""""+start_timestamp+"""\" AS TIMESTAMP)
  AND """+"CAST(\""+end_timestamp+"\" AS TIMESTAMP)"""

query = query + """
GROUP BY
  period_start
ORDER BY
  period_start DESC"""

if verbose:
  print(query)
df = pandas_gbq.read_gbq(query, project_id=project_id)

**Step 5**

Create jobs table that later will be used to chart the number of jobs that ran in each time period along with the foregraoud/background jobs breakdown.

In [ ]:
query = "create table if not exists "+dataset+""".jobs as SELECT
  timeline.job_id,
  timestamp_trunc(timeline.period_start, MINUTE) period_start,
  jobs.user_email,
  jobs.Job_type,
  CASE
    WHEN CONTAINS_SUBSTR(jobs.user_email, 'gserviceaccount') THEN 'background'
    ELSE 'foreground'
END
  AS query_type
FROM
    `"""+project_id+".region-"+region+""".INFORMATION_SCHEMA.JOBS_TIMELINE` AS timeline
  JOIN
    `"""+project_id+".region-"+region+""".INFORMATION_SCHEMA.JOBS` AS jobs
  ON
    timeline.job_id = jobs.job_id
WHERE
  timeline.period_start BETWEEN CAST("""+"\""+start_timestamp+"\""+""" AS TIMESTAMP)
  AND CAST("""+"\""+end_timestamp+"\""+" AS TIMESTAMP)"

if verbose:
  print(query)
df = pandas_gbq.read_gbq(query, project_id=project_id)

**Step 6**<br>
We create a dynamic query that creates buckets incremented by 50 from 0 to the highest number of slots used during the selected time period.<br>
The original time periods are seconds, so we aggregate the data in Minute granularity and for each minute we take the high<br> watermark of slot usage (since BigQuery autoscaler's minimum is 1 minute).<br><br>
Then we assign each minute to a specific bucket according to the maximum slots that ut used.<br>
We create a view called bucketed to hold the result and enable further calculations.

In [ ]:
query1 = "select max(total_slot_ms) as max_slots from "+dataset+".slots"
df = pandas_gbq.read_gbq(query1, project_id=project_id)
max_slots = int(df._get_value(0, 'max_slots'))
query2 = "create view if not exists "+dataset+".bucketed"+""" as SELECT
  timestamp_trunc(period_start, MINUTE) as period_start,
  sum(total_queries) as total_queries,
  max(case
when total_slot_ms = 0 then 0\n"""
for i in range(0, max_slots, 50):
  line = "when total_slot_ms between "+str(i)+" and "+str(i+50)+" then "+str(i+50)+'\n'
  query2 = query2 + line
query2 = query2 + """  end) as bucket
FROM """+dataset+""".slots
  group by timestamp_trunc(period_start, MINUTE)"""

if verbose:
  print(query2)

df = pandas_gbq.read_gbq(query2, project_id=project_id)

**Step 7**<br>
Here we create a view called buckets_count that shows hoe many time periods (minutes) fall into each bucket.<br>
Then we print the contents of the view.

In [ ]:
query = "create view if not exists "+dataset+""".buckets_count as SELECT
  bucket,
  COUNT(*) as periods,
FROM `"""+project_id+"."+dataset+""".bucketed`
where bucket is not null
GROUP BY
  bucket"""

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)
query = 'select * from '+dataset+".buckets_count order by bucket"

if verbose:
  print(query)
df = pandas_gbq.read_gbq(query, project_id=project_id)
print(df)

**Step 8**

INFORMATION_SCHEMA.JOBS_TIMELINE and INFORMATION_SCHEMA.JOBS contain rows from time periods when we had BigQuery activity. Any time when no queries ran at all will not be represented there and this may distort the results (The price will not change but the charts will look different).
So we want to create another table/view that will contain also all the "silent" times with a count of zero queries.
We iterate over all the minutes from start_date to end_date and if this specific minute is not present in the slots table then we add it to an intermediate table "silent_periods".
Once finished, we create a view called "slots_full" that contains both the silent times and the used times.


We do the same with jobs table for jobs table, where we create views job_types and query_types adding zero records when there was no activity.

Those "padded" views will be used later when we create charts.

In [ ]:
periods = []
slots = []
queries = []

query = "select period_start from "+dataset+".slots"
df = pandas_gbq.read_gbq(query, project_id=project_id)

minutes = pd.date_range(start =start_timestamp,
         end =end_timestamp, freq ='min')

for minute in minutes:
  if minute not in df['period_start'].values:
      periods.append(minute)
      slots.append(0)
      queries.append(0)

new_df = pd.DataFrame({
    "period_start": periods,
    "total_queries": queries,
    "total_slot_ms": slots,
})

pandas_gbq.to_gbq(new_df, dataset+".silent_periods", project_id=project_id, if_exists='replace',)

query = """CREATE VIEW if not exists
  """+dataset+""".slots_full AS
SELECT
  timestamp(period_start) as period_start, cast(total_queries as int) as total_queries, cast(total_slot_ms as int) as total_slot_ms
FROM
  """+dataset+""".slots
UNION ALL
SELECT
  timestamp(period_start) as period_start, cast(total_queries as int) as total_queries, cast(total_slot_ms as int) as total_slot_ms
FROM
  """+dataset+".silent_periods"

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

In [ ]:
query = """SELECT
  TIMESTAMP_TRUNC(period_start, HOUR) AS hour,
  SUM(foreground) AS fg_count,
  SUM(background) AS bg_count
FROM (
  SELECT
    period_start,
    CASE query_type
      WHEN 'foreground' THEN 1
      ELSE 0
  END
    AS foreground,
    CASE query_type
      WHEN 'background' THEN 1
      ELSE 0
  END
    AS background
  FROM
    """+dataset+""".jobs)
GROUP BY
  hour
ORDER BY
  hour"""

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

# Adding "empty" hours
hours = pd.date_range(start =start_timestamp,
         end =end_timestamp, freq ='h')
periods = []
fg_count = []
bg_count = []
for hour in hours:
  if hour not in df['hour'].values:
      periods.append(hour)
      fg_count.append(0)
      bg_count.append(0)

new_df = pd.DataFrame({
    "hour": periods,
    "fg_count": fg_count,
    "bg_count": bg_count,
})

df = pd.concat([df, new_df])
pandas_gbq.to_gbq(df, dataset+".query_types", project_id=project_id, if_exists='replace',)



In [ ]:
query = """select
  TIMESTAMP_TRUNC(period_start, HOUR) AS hour,
  SUM(query) AS query,
  SUM(load_) AS load,
  sum(extract_) as extract_,
  sum(null_) as null_
FROM (
  SELECT
    period_start,
    CASE job_type
      WHEN 'QUERY' THEN 1
      ELSE 0
    END
    AS query,
    CASE job_type
      WHEN 'LOAD' THEN 1
      ELSE 0
    END AS load_,
    CASE job_type
      WHEN 'EXTRACT' THEN 1
      ELSE 0
    END AS extract_,
    CASE job_type
      WHEN 'COPY' THEN 1
      ELSE 0
    END AS copy,
    CASE job_type
      WHEN 'NULL' THEN 1
      ELSE 0
    END AS null_
  FROM """ +dataset+""".jobs)
GROUP BY
  hour
ORDER BY
  hour"""

if verbose:
   print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

# Adding "empty" hours
hours = pd.date_range(start =start_timestamp,
         end =end_timestamp, freq ='h')
periods = []
query = []
load = []
extract_ = []
null_ = []
for hour in hours:
  if hour not in df['hour'].values:
      periods.append(hour)
      query.append(0)
      load.append(0)
      extract_.append(0)
      null_.append(0)

new_df = pd.DataFrame({
    "hour": periods,
    "query": query,
    "load": load,
    "extract_": extract_,
    "null_": null_,
})

df2 = pd.concat([df, new_df])
pandas_gbq.to_gbq(df2, dataset+".job_types", project_id=project_id, if_exists='replace',)

For our next calculations we need to know Two numbers:


1.   How many hours there are in our timeframe
2.   How many time periods we have

So steps 9 and 10 find those values.





**Step 9**<br>
Calculate how many hours we have in the chosen time frame.

In [ ]:
from datetime import datetime
date_format = '%Y-%m-%d'
diff = datetime.strptime(end_timestamp,date_format) - datetime.strptime(start_timestamp, date_format)
hours = diff.days * 24 + diff.seconds / 3600
if verbose:
  print("hours: "+str(hours))

**Step 10**<br>
Calculate how many time slots we have.

In [ ]:
query = "select count(*) as periods from "+dataset+".bucketed"
df = pd.io.gbq.read_gbq(query=query, project_id=project_id, dialect='standard')
periods = df._get_value(0, 'periods')
if verbose:
  print("periods: "+str(periods))

**Step 11**<br>
This is the heart of our calculation.<br><br>
We know how many hours were in the selected timeframe and how many time periods we had during this timeframe.
So we can calculate what is the percentage of all the time that we used each bucket.<br>
If we know the percentage of time and we know how many wours we had in the timeframe then we can calculate how many hours each bucket was active.<br>
And we know how much each slot/hour costs for each BigQuery edition so we can calculate the cost each bucket incured.

So here we create a view called calculated that holds the bucket, the percentage of all time that this bucket was used, how many hours it was used, and how much we would be charged for it if we use the selected edition.

In [ ]:
query = "create view if not exists "+dataset+""".calculated as SELECT
  bucket,
  round(periods/"""+str(periods)+"""*100, 2) as percentage,
  round(periods/"""+str(periods)+"*"+str(hours)+""", 2) as hours,
  round(periods/"""+str(periods)+"*"+str(hours)+"*bucket*"+str(se_price)+""", 2) as cost_usd_se,
  round(periods/"""+str(periods)+"*"+str(hours)+"*bucket*"+str(ee_price)+""", 2) as cost_usd_ee,
  round(periods/"""+str(periods)+"*"+str(hours)+"*bucket*"+str(eep_price)+""", 2) as cost_usd_eep
FROM
  `"""+project_id+"."+dataset+""".buckets_count`
  order by bucket"""

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

**Let's see how it looks like.**

In [ ]:
query = "select * from "+dataset+".calculated"
calculated = pandas_gbq.read_gbq(query, project_id=project_id)

In [ ]:
print(calculated)

**Stage 12**<br>
Here is a visualization of the histogram that shows the distribution of the slot buckets and the time spent in each of them.<br>
There should be a "sweet spot" where most of the buckets under it are heavily used and above it there is only slight usage.<br>
This shhould give you the sense of where that sweet spot should be.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

calculated.plot(x="bucket", y="percentage", kind="line",figsize=(15,9))

**Step 13**

The following chart shows how many queries ran on each time during our test period.

In [ ]:
query = "select period_start, total_queries from "+dataset+".slots_full order by period_start"
df = pandas_gbq.read_gbq(query, project_id=project_id)
df.plot(x="period_start", y="total_queries", kind="line",figsize=(25,9))

**Step 14**<br>
Use the following query to identify where slot usage drops below 1% of the time.We will set this bucket as the max_slots as we can probably live with less than 1% of the queries that will get slower.

In [ ]:
query = "select min(bucket) as recommended from "+dataset+".calculated where percentage < 1 and bucket > 0"

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)
print(df)

**Step 15**<br>
The following steps try to find the optimal max_slots that best balances cost and performance.<br>
As a first step choose the max_slots value you want to check.

In [ ]:
max_slots = 100 # @param {"type":"number"}

**Explanation**<br>
The trade off is cost vs. performance. If we choose the right max_slots than we can reduce the cost while only a small number of queries will decrease in performance. If these are not time critical queries (such as ETLs, background jobs etc.) than we may want to "sacrifice" them in return for  lower cost.

**Step 16**<br>
We try to calculate how much will we pay if we choose the above max_slots.<br>
We also add the "tail" which are all the buckets that were used less than 1% of the time and now will all accumulate at the max_slots bucket.

In [ ]:
answer = calculated[calculated['bucket'] <= max_slots]
se = answer['cost_usd_se'].sum()
ee = answer['cost_usd_ee'].sum()
eep = answer['cost_usd_eep'].sum()

query = "select cost_usd from "+dataset+".bytes"
df = pandas_gbq.read_gbq(query, project_id=project_id)
cost_bytes = str(round(df["cost_usd"][0]))

# Now adding all the percent fragments that were used less than 1% and now will be added to the max_slots bucket
tail = calculated[calculated['percentage'] < 1]
tail_cost_se = tail['percentage'].sum() * se_price * max_slots
tail_cost_ee = tail['percentage'].sum() * ee_price * max_slots
tail_cost_eep = tail['percentage'].sum() * eep_price * max_slots

print('Total cost estimation for the time frame using Standard edition and maximum slots '+str(max_slots)+' is '+str(round(se + tail_cost_se))+' USD.' )
print('Total cost estimation for the time frame using Enterprise edition and maximum slots '+str(max_slots)+' is '+str(round(ee + tail_cost_ee))+' USD.' )
print('Total cost estimation for the time frame using Enterprise plus eition and maximum slots '+str(max_slots)+' is '+str(round(eep + tail_cost_eep))+' USD.' )
print('Running the same workload with on-demand pricing costed you '+cost_bytes+" USD")

**Step 17**<br>
We want to see which queries will be the most affected by the slot decrease (the ones that has slot consumption above max_slots).
Many times you find out that those queries are not time sensitive (such as background processes) and you can sacrifice their performance to lower cost.

In [ ]:
query = """SELECT
  *
FROM (
  SELECT
    timeline.job_id AS job_id,
    jobs.query AS query,
    jobs.job_type AS job_type,
    ROUND(MAX(timeline.period_slot_ms/1000)) AS total_slot_ms,
    COUNT(timeline.job_id) AS slices
  FROM
    `"""+project_id+"`.`region-"+region+"""`.INFORMATION_SCHEMA.JOBS_TIMELINE AS timeline
  JOIN
    `"""+project_id+"`.`region-"+region+"""`.INFORMATION_SCHEMA.JOBS AS jobs
  ON
    timeline.job_id = jobs.job_id
  WHERE
    period_start BETWEEN CAST(\""""+start_timestamp+"""\" AS TIMESTAMP)
  AND """+"CAST(\""+end_timestamp+"\" AS TIMESTAMP)"""+"""
  GROUP BY
    job_id,
    query,
    job_type
  ORDER BY
    slices DESC)
WHERE
  total_slot_ms>"""+str(max_slots)

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)
print(df)


**Step 18**

Here we calculate what is the optimal baseline slots value, if any.

In [ ]:
query = """SELECT
  *
FROM
  """+dataset+""".calculated
WHERE
  percentage>60
  and bucket>0"""

if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)
if df.empty:
  print('It is recommended not to set a baseline.')
else:
  bucket = df['bucket'][0]
  cost_se = df['cost_usd_se'][0]
  cost_ee = df['cost_usd_ee'][0]
  cost_eep = df['cost_usd_eep'][0]
  percentage = df['percentage'][0]
  if percentage > 60 and percentage < 80 :
    print("Setting a baseline of "+str(bucket)+""" with 3 Year commitment will save you this much for the tested period:
     With Standard edition: """+str(cost_se*0.4)+""" dollars
     With Enterprise edition: """+str(cost_ee*0.4)+""" dollars
     With Enterprise plus edition: """+str(cost_eep*0.4)+" dollars")
  if percentage >= 80 :
    print("Setting a baseline of "+str(bucket)+""" with 1 Year commitment will save you this much for the tested period:
     With Standard edition: """+str(round(cost_se*0.2))+""" dollars
     With Enterprise edition: """+str(round(cost_ee*0.2))+""" dollars
     With Enterprise plus edition: """+str(round(cost_eep*0.2))+" dollars")

**Step 19**

Here we create a chart that shows how many queries ran during our test period. We also break them down to background queries that were scheduled or triggered by automatic tools such as cloud scheduler, Composer etc (and hence are less time sensitive) and foreground queries that were run interactively by human users.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

query = "SELECT hour, fg_count, bg_count FROM "+dataset+".query_types  ORDER BY hour"
if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

# Plotting the stacked bar chart
fig, ax = plt.subplots(figsize=(33,7))

ax.bar(df['hour'], df['bg_count'], label='Background', width = 0.03)
ax.bar(df['hour'], df["fg_count"], bottom=df['bg_count'], label='Foreground', width = 0.03)

# Adding labels and title
ax.set_xlabel('Hour')
ax.set_ylabel('Queries')
ax.set_title('Breakdown of query types per hour')
ax.legend()


# Show the plot
plt.show()

cnt_bg = df["bg_count"].sum()
cnt_fg = df["fg_count"].sum()
total = cnt_fg + cnt_bg
print (str(round(cnt_bg/(total/100),2))+" percent of the queries were background queries and "+str(round(cnt_fg/(total/100),2))+ " were foreground queries")


**Step 20**

Here we do pretty much the same as we did in the previous step, only instead of grouping the queries into foreground/background we divide by the job types (query, load, extract and null).

Null is used for internal queries like materialized vies refresh.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

query = "SELECT hour, query,load,extract_,null_ FROM "+dataset+".job_types  ORDER BY hour"
if verbose:
  print(query)

df = pandas_gbq.read_gbq(query, project_id=project_id)

# Plotting the stacked bar chart
fig, ax = plt.subplots(figsize=(33,7))

ax.bar(df['hour'], df['query'], label='Query', width = 0.03)
ax.bar(df['hour'], df["load"], bottom=df['query'], label='Load', width = 0.03)
ax.bar(df['hour'], df["extract_"], bottom=df['load'], label='Extract', width = 0.03)
ax.bar(df['hour'], df["null_"], bottom=df['extract_'], label='Null', width = 0.03)

# Adding labels and title
ax.set_xlabel('Hour')
ax.set_ylabel('Queries')
ax.set_title('Breakdown of job types per hour')
ax.legend()


# Show the plot
plt.show()

cnt_query = df["query"].sum()
cnt_load = df["load"].sum()
cnt_extract = df["extract_"].sum()
cnt_null = df["null_"].sum()

total = cnt_query+cnt_load+cnt_extract+cnt_null

print (str(round(cnt_query/(total/100),2))+" percent of the jobs were query jobs, "+str(round(cnt_load/(total/100),2))+ " were load jobs,")
print (str(round(cnt_extract/(total/100),2))+" percent of the jobs were extract jobs and "+str(round(cnt_null/(total/100),2))+ " were null (internal) jobs.")

**Step 21**<br>
Cleanup

In [ ]:
queries = []
queries.append("drop view "+dataset+".calculated")
queries.append("drop view "+dataset+".buckets_count")
queries.append("drop view "+dataset+".bucketed")
queries.append("drop table "+dataset+".bytes")
queries.append("drop table "+dataset+".slots")
queries.append("drop table "+dataset+".job_types")
queries.append("drop table "+dataset+".query_types")
queries.append("drop table "+dataset+".jobs")
queries.append("drop view "+dataset+".slots_full")
queries.append("drop table "+dataset+".silent_periods")

for query in queries:
  df = pandas_gbq.read_gbq(query, project_id=project_id)

print ("Delete complete.")